# Actions de configuration
- Créer un répertoire image-heberg.fr dans son drive
- Créer un fichier key.web contenant la clef `_KEY_FOR_IA_TRAINING_` configurée dans le fichier config.php

Sources :
  - [Coder un réseau de neurones convolutifs de classification d'image avec Python et Tensorflow](https://www.youtube.com/watch?v=6FHtTyZxS5s)
  - [Notebook Jupyter associé](https://github.com/anisayari/Youtube-apprendre-le-deeplearning-avec-tensorflow/blob/master/%234%20-%20CNN/A_deep_introduction_to_CNN%20(1).ipynb)
  - [Connexion à Google Drive](https://colab.research.google.com/drive/1znbKFQs98XolesqOZzseFjOLVPkGoXiq?usp=sharing#scrollTo=RIz91NnKbq-j)
  - [TensorflowJS](https://www.tensorflow.org/js/tutorials/conversion/import_keras?hl=fr)

# Charger toutes les libs et modules requis pour le projet

In [ ]:
# Infos sur l'environnement
!apt install pciutils --quiet
!lspci | grep -i nvidia
!uname -m && cat /etc/*release

# Installation de cuda
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
!sudo dpkg -i cuda-keyring_1.1-1_all.deb
!apt install cuda-toolkit cuda-drivers cudnn

# Vérifier l'installation de cuda
!wget https://github.com/NVIDIA/cuda-samples/archive/refs/heads/master.zip
!unzip master.zip
!apt install cmake
!python3 -m pip install cmake
!cd cuda-samples-master && make
!./cuda-samples-master/bin/x86_64/linux/release/deviceQuery

# Forcer le passage à Python >=3.11 pour pouvoir utiliser hashlib.file_digest() (actuellement Python3.10)
!python3 --version
import sys, os
print(sys.version)

# Adapté de https://stackoverflow.com/a/74538231
#install python 3.11 and dev utils
#you may not need all the dev libraries, but I haven't tested which aren't necessary.
!sudo apt-get update -y
!sudo apt-get install python3.11 python3.11-dev python3.11-distutils libpython3.11-dev python3.11-venv binfmt-support

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1

# install pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall

#install colab's dependencies
!python3 -m pip install ipython traitlets jupyter psutil matplotlib setuptools ipython_genutils ipykernel jupyter_console notebook prompt_toolkit httplib2 astor

# Installation des dépendances de la suite de mes scripts
!python3 -m pip install opencv-python

# link to the old google package
!ln -s /usr/local/lib/python3.10/dist-packages/google /usr/local/lib/python3.11/dist-packages/google

# There has got to be a better way to do this...but there's a bad import in some of the colab files
# IPython no longer exposes traitlets like this, it's a separate package now
!sed -i "s/from IPython.utils import traitlets as _traitlets/import traitlets as _traitlets/" /usr/local/lib/python3.11/dist-packages/google/colab/*.py
!sed -i "s/from IPython.utils import traitlets/import traitlets/" /usr/local/lib/python3.11/dist-packages/google/colab/*.py

# Vérifier les versions
!python3 --version
import sys, os
print(sys.version)

#restart environment so you don't have to do it manually
os.kill(os.getpid(), 9)

In [ ]:
# Vérifier les versions
!python3 --version
import sys, os
print(sys.version)

# Vérifier qu'on voit bien le GPU
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI/lib64
!nvidia-smi

# Import de toutes les libs requises dans le projet
import datetime
import os
import sys
import time
!python3 -m pip install numpy tensorflow[and-cuda] tensorflowjs --use-deprecated=legacy-resolver
# Tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow as tf
# TensorFlowJS (pour l'export du modèle)
import tensorflowjs as tfjs
# Affichage des images
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
# Réseaux
from http.client import IncompleteRead
import requests
from urllib.parse import urlparse
from urllib.request import urlretrieve, URLError
# Enregistrer sur le Google Drive associé
from google.colab import drive
from google.colab import files
# Chiffrement des fichiers sur le Google Drive (confidentialité + CGU)
!python3 -m pip install cryptography --quiet
from cryptography.fernet import Fernet
# Vérification des images
import hashlib

print(sys.version)
print('TensorFlow :',tf.__version__)
tf.test.gpu_device_name()

# Gestion des stockages
- Définition des constantes
- Copie dans l'environnement des fichiers en cache chiffré sur Google Drive

In [ ]:
# Monter le drive
drive.mount('/content/drive', force_remount=True)

# Définir les stockages
data_persistent_dir_base = '/content/drive/MyDrive/image-heberg.fr/'
data_persistent_key = data_persistent_dir_base + 'key.fernet'
data_persistent_key_web = data_persistent_dir_base + 'key.web'
data_crypted_persistent_dir_images = data_persistent_dir_base + 'images/'
data_persistent_dir_models = data_persistent_dir_base + 'models/'
data_versatile_dir_images = '/data/images/'

# Catégories d'images
cat_image_approuvee = 'approuvee'
cat_image_bloquee = 'bloquee'


# Création d'un dossier si nécessaire
def create_folder_if_required(path, name):
    folder = path + name
    if not os.path.exists(folder):
        os.makedirs(folder)
        print('Création du dossier', folder)


# Création des dossiers si nécessaire
create_folder_if_required(data_versatile_dir_images, cat_image_approuvee)
create_folder_if_required(data_versatile_dir_images, cat_image_bloquee)
create_folder_if_required(data_crypted_persistent_dir_images, cat_image_approuvee)
create_folder_if_required(data_crypted_persistent_dir_images, cat_image_bloquee)

# Création de la clef de chiffrement si nécessaire
if not os.path.isfile(data_persistent_key):
    key = Fernet.generate_key()
    print('Génération de la clef de chiffrment pour Google Drive', key)
    file = open(data_persistent_key, 'wb')
    file.write(key)
    file.close()

# Chargement de la clef de chiffrement
file = open(data_persistent_key, 'rb')
encryption_key = file.read()
file.close()


# Déchiffrer et copier des fichiers
def uncrypt_and_copy(src_file, dst_file, encryption_key):
    if not os.path.isfile(src_file):
        print('Fichier inexistant :', src_file)
        return
    # Si le fichier dst existe déjà, ne pas redéchiffrer
    if os.path.isfile(dst_file):
        return

    # Lire le fichier source
    file = open(src_file, 'rb')
    src_content = file.read()
    file.close()

    # Chargement de la clef de chiffrement
    f = Fernet(encryption_key)

    # Ecrire dans le fichier de destination
    file = open(dst_file, 'wb')
    file.write(f.decrypt(src_content))
    file.close()
    print(src_file, '=>', dst_file)


# Copier en local les données chiffrées du Drive
# Images approuvées
for image in os.scandir(data_crypted_persistent_dir_images + cat_image_approuvee):
    uncrypt_and_copy(data_crypted_persistent_dir_images + cat_image_approuvee + '/' + image.name,
                     data_versatile_dir_images + cat_image_approuvee + '/' + image.name, encryption_key)

# Images bloquées
for image in os.scandir(data_crypted_persistent_dir_images + cat_image_bloquee):
    uncrypt_and_copy(data_crypted_persistent_dir_images + cat_image_bloquee + '/' + image.name,
                     data_versatile_dir_images + cat_image_bloquee + '/' + image.name, encryption_key)

# Nombres de fichiers
print(cat_image_approuvee, ':', len([entry for entry in os.listdir(data_versatile_dir_images + cat_image_approuvee)]))
print(cat_image_bloquee, ':', len([entry for entry in os.listdir(data_versatile_dir_images + cat_image_bloquee)]))

# Récupération des images bloquées / approuvées
- Téléchargement de la liste des images depuis image-heberg.fr
- Récupération des images non en cache
- Mise en cache des images sur Google Drive avec un stockage chiffré (respect des CGU)


In [ ]:
# Télécharger des images à partir d'une URL de liste d'images avec vérification du md5 du fichier
def downloadListeImageFromURL(URL_liste_images, categorie_image, encryption_key):
    nb_images_total = 0
    nb_images_dl = 0
    liste_images = requests.get(URL_liste_images, stream=True).raw
    for une_image in liste_images:
        [img_md5, img_url] = une_image.strip().decode('utf-8').split('###', 1)
        img_url_parsee = urlparse(img_url)
        img_name = os.path.basename(img_url_parsee.path)
        img_dest_versatile = data_versatile_dir_images + categorie_image + '/' + img_name

        # Si l'image est déjà présente (en cache via le Drive), ne pas la re-télécharger
        if not os.path.isfile(img_dest_versatile):
            img_dest_persistent = data_crypted_persistent_dir_images + categorie_image + '/' + img_name
            print('Téléchargement de', img_name, '=>', img_dest_versatile)

            try:
                urlretrieve(img_url, img_dest_versatile)
            except (IncompleteRead, ConnectionResetError, URLError) as e:
                # OVH limite le nombre de requêtes sur une volumétrie / durée
                # => En cas de fermeture de la connexion, faire une pause
                print('Exception sur les données reçues', e)
                time.sleep(45)
                continue  # continue to next row

            # L'ajouter au cache chiffré (Google Drive)
            # Lire le fichier source
            file = open(img_dest_versatile, 'rb')
            src_content = file.read()
            file.close()

            # Chargement de la clef de chiffrement
            f = Fernet(encryption_key)

            # Ecrire dans le fichier de destination
            file = open(img_dest_persistent, 'wb')
            file.write(f.encrypt(src_content))
            file.close()

            # Incrémenter le compteur
            nb_images_dl += 1

        ## DEBUG - Afficher l'image ##
        #import matplotlib.pyplot as plt
        #import matplotlib.image as mpimg
        #img = mpimg.imread(img_dest_versatile) #Replace "image.jpg" with the path of your image
        #plt.imshow(img)
        #plt.axis('off')
        #plt.show()


        # Vérification du hash de l'image
        file = open(img_dest_versatile, 'rb')
        file_md5 = hashlib.file_digest(file, 'md5')
        file.close()
        if not img_md5 == file_md5.hexdigest():
            print('Erreur de hash pour', img_name, ': calculé', file_md5.hexdigest(), '- attendu :', img_md5)
            os.remove(file)

        # Incrémenter le compteur
        nb_images_total += 1

    print(categorie_image, ': ', nb_images_total, '(dont', nb_images_dl, 'téléchargées)')


# Récupération de la clef pour communiquer avec image-heberg.fr
# Lire le fichier source
file = open(data_persistent_key_web, 'r')
key_web = file.read()
file.close()

# Images bloquées
downloadListeImageFromURL('https://www.image-heberg.fr/ia.php?etat=bloque&key=' + key_web, cat_image_bloquee,
                          encryption_key)
# Images approuvées
downloadListeImageFromURL('https://www.image-heberg.fr/ia.php?etat=approuvee&key=' + key_web, cat_image_approuvee,
                          encryption_key)


# Vérification de la qualité des données

In [ ]:
yourDirectory = data_crypted_persistent_dir_images + cat_image_approuvee + '/'
for filename in os.scandir(yourDirectory):
  if filename.name.endswith(".jpg"):
    print(yourDirectory+filename.name)
    cv2.imread(yourDirectory+filename.name)
yourDirectory = data_crypted_persistent_dir_images + cat_image_bloquee + '/'
for filename in os.scandir(yourDirectory):
  if filename.name.endswith(".jpg"):
    print(yourDirectory+filename.name)
    cv2.imread(yourDirectory+filename.name)

In [ ]:
from struct import unpack

marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()

    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]
            if len(data)==0:
                break


bads = []

# Images approuvées
for image in os.scandir(data_crypted_persistent_dir_images + cat_image_approuvee):
    try:
        mon_image = JPEG()
        image.decode()
    except:
        bads.append(img)
    uncrypt_and_copy(data_crypted_persistent_dir_images + cat_image_approuvee + '/' + image.name,
                     data_versatile_dir_images + cat_image_approuvee + '/' + image.name, encryption_key)

# Images bloquées
for image in os.scandir(data_crypted_persistent_dir_images + cat_image_bloquee):
    uncrypt_and_copy(data_crypted_persistent_dir_images + cat_image_bloquee + '/' + image.name,
                     data_versatile_dir_images + cat_image_bloquee + '/' + image.name, encryption_key)

for img in tqdm(images):
  image = osp.join(root_img,img)
  image = JPEG(image)



for name in bads:
  print(name)
  #os.remove(osp.join(root_img,name))

# Génération des jeux d'entrainement et de validation

In [ ]:
# Traitements des images par lots pour accélérer le traitement
batch_size = 5
# Taille des images sur lequel le traitement sera effectué (plus elle est grande, plus le temps de calcul est long)
img_height = 200
img_width = 200
# Taille de l'échantillon de validation (0 - 1)
validation = 0.2
# Avoir des traces d'erreur verbeuses
#tf.debugging.disable_traceback_filtering()

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_versatile_dir_images,
    validation_split=validation,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_versatile_dir_images,
    validation_split=validation,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Valeurs possibles en sortie
class_names = val_data.class_names
print(class_names)

# Affichage d'un sample des images

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_data.take(1):
    for i in range(batch_size):
        ax = plt.subplot(1, batch_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

# Définition et entraînement du CNN

In [ ]:
# Nombre de classes possibles (Approuvée, Bloquée)
num_classes = 2
# Nombre de tours d'entrainement du modele
nb_epochs = 10

model = tf.keras.Sequential([
    layers.Rescaling(1. / 255),
    layers.Conv2D(128, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'], )

logdir = "logs"

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, write_images=logdir)

model.fit(
    train_data,
    validation_data=val_data,
    epochs=nb_epochs,
    callbacks=[tensorboard_callback]
)

In [ ]:
model.summary()

# Interface de test du CNN généré

In [ ]:
file_to_predict = files.upload()
for file_ in file_to_predict:
    image_to_predict = cv2.imread(file_, cv2.IMREAD_COLOR)
    plt.imshow(cv2.cvtColor(image_to_predict, cv2.COLOR_BGR2RGB))
    plt.show()
    img_to_predict = np.expand_dims(cv2.resize(image_to_predict, (200, 200)), axis=0)
    predict_x = model.predict(img_to_predict)
    res = np.argmax(predict_x, axis=1)
    # [proba Approuvée, proba Bloquée]
    print(predict_x)
    print(res)

    if res == 1:
        print("BLOQUEE")
    elif res == 0:
        print("APPROUVEE")

# Export du modèle vers tensorflowjs

In [ ]:
tfjs.converters.save_keras_model(model, data_persistent_dir_models)

# Dashboard des logs du CNN

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs